<h1 align='center'>Mask Detection using python openCV

_Importing the necessary libraries_

In [1]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
 

_Frontal face recognition using Haar Classifier of keras_

In [2]:
cascPath = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)


_Loading the pre-trained model which classifies mask/without mask_

In [3]:
model = load_model("mask_detector_model.h5")

In [ ]:
 
video_capture = cv2.VideoCapture(0)

while True:
    try:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
        faces_scaled=[]
        preds=[]
        for (x, y, w, h) in faces:
            face_frame = frame[y:y+h,x:x+w]
            face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
            face_frame = cv2.resize(face_frame, (224, 224))
            face_frame = img_to_array(face_frame)
            face_frame = np.expand_dims(face_frame, axis=0)
            face_frame =  preprocess_input(face_frame)
            faces_scaled.append(face_frame)

        for i in faces_scaled:
            prediction = model.predict(i)
            preds.append(prediction)
            
        for i,(x,y,w,h) in enumerate(faces):
            (mask, withoutMask) = preds[i][0]
            
            
            if mask > withoutMask :
                label = "Mask on" 
                color = (0, 255, 0) 
            else:
                label = "Please wear a Mask"
                color = (0, 0, 255)
    
            cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
    
            cv2.putText(frame, label, (x, y- 10),cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
 
            
        
        cv2.imshow('Video', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break
    except:
        pass

video_capture.release()
cv2.destroyAllWindows()